<a href="https://colab.research.google.com/github/diegotluz/projects-colab/blob/main/is_it_a_bird_creating_a_model_from_your_own_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Isso é um passaro?

In [ ]:
#NB: Kaggle requer verificação por telefone para usar a Internet ou uma GPU. Se você ainda não fez isso, a célula abaixo falhará
# Este código está aqui apenas para verificar se sua internet está habilitada. Não faz mais nada.
# Aqui está um tópico de ajuda sobre como verificar seu número de telefone: https://www.kaggle.com/product-feedback/135367

import socket,warnings
try:
    socket.setdefaulttimeout(1)
    socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect(('1.1.1.1', 53))
except socket.error as ex: raise Exception("STOP: No internet. Click '>|' in top right and set 'Internet' switch to on")

In [ ]:
# É uma boa ideia garantir que você esteja executando a versão mais recente de todas as bibliotecas necessárias.
# `!pip install -Uqq <libraries>` atualiza para a versão mais recente de <libraries>
# NB: Você pode ignorar com segurança quaisquer avisos ou erros que pip cuspa sobre a execução como root ou incompatibilidades
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -Uqq fastai duckduckgo_search

In [ ]:
!python -m pip  install -U pip

!pip install -Uqq fastai duckduckgo_search

Em 2015, a ideia de criar um sistema de computador que pudesse reconhecer pássaros foi considerada tão escandalosamente desafiadora que foi a base [desta piada do XKCD](https://xkcd.com/1425/):

<!-- ![image.png](attachment:a0483178-c30e-4fdd-b2c2-349e130ab260.png) -->

Mas hoje podemos fazer exatamente isso, em apenas alguns minutos, utilizando recursos totalmente gratuitos!

As etapas básicas que tomaremos são:

1. Use DuckDuckGo para pesquisar imagens de "fotos de pássaros"
1. Use DuckDuckGo para pesquisar imagens de "fotos da floresta"
1. Ajuste uma rede neural pré-treinada para reconhecer esses dois grupos
1. Experimente executar este modelo na imagem de um pássaro e veja se funciona.

## Etapa 1: Baixe imagens de pássaros e não pássaros

In [ ]:
from duckduckgo_search import DDGS
from fastcore.all import *

def search_images(term, max_images=30):
    print(f"Searching for '{term}'")
    with DDGS() as ddgs:
        # gerador que produz dicts com:
        # {'title','image','thumbnail','url','height','width','source'}
        search_results = ddgs.images(keywords=term)
        # grap número de URLs max_images
        image_urls = [result.get("image") for result in search_results[:max_images]]
        # converter para L (classe de lista funcionalmente estendida de fastai)
        return L(image_urls)

Vamos começar procurando uma foto de um pássaro e ver que tipo de resultado obtemos. Começaremos obtendo URLs de uma pesquisa:

In [ ]:
#NB: `search_images` depende de duckduckgo.com, que nem sempre retorna respostas corretas.
# Se você receber um erro JSON, tente executá-lo novamente (podem ser necessárias algumas tentativas).
urls = search_images('bird photos', max_images=1)
urls[0]

... e depois baixe um URL e dê uma olhada nele:

In [ ]:
from fastdownload import download_url
dest = 'bird.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

Agora vamos fazer o mesmo com as "fotos da floresta":

In [ ]:
download_url(search_images('forest photos', max_images=1)[0], 'forest.jpg', show_progress=False)
Image.open('forest.jpg').to_thumb(256,256)

Nossas pesquisas parecem estar dando resultados razoáveis, então vamos pegar alguns exemplos de fotos de "pássaros" e "florestas" e salvar cada grupo de fotos em uma pasta diferente (também estou tentando pegar uma variedade de condições de iluminação aqui):

In [ ]:
searches = 'forest','bird'
path = Path('bird_or_not')
from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photo'))
    sleep(10)  #Faça uma pausa entre as pesquisas para evitar sobrecarregar o servidor
    download_images(dest, urls=search_images(f'{o} sun photo'))
    sleep(10)
    download_images(dest, urls=search_images(f'{o} shade photo'))
    sleep(10)
    resize_images(path/o, max_size=400, dest=path/o)

##Etapa 2: treinar nosso modelo

Algumas fotos podem não ser baixadas corretamente, o que pode causar falha no treinamento do modelo, portanto, iremos removê-las:

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

Para treinar um modelo, precisaremos de `DataLoaders`, que é um objeto que contém um *conjunto de treinamento* (as imagens usadas para criar um modelo) e um *conjunto de validação* (as imagens usadas para verificar a precisão de um modelo - não usado durante o treinamento). Em `fastai` podemos criar isso facilmente usando um `DataBlock` e visualizar imagens de amostra dele:

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=6)

Aqui está o que cada um dos parâmetros `DataBlock` significa:

    blocks=(ImageBlock, CategoryBlock),

As entradas do nosso modelo são imagens e as saídas são categorias (neste caso, "bird" or "forest").

    get_items=get_image_files,

Para encontrar todas as entradas do nosso modelo, execute a função `get_image_files` (que retorna uma lista de todos os arquivos de imagem em um caminho).

    splitter=RandomSplitter(valid_pct=0.2, seed=42),

Divida os dados em conjuntos de treinamento e validação aleatoriamente, usando 20% dos dados para o conjunto de validação.

    get_y=parent_label,

Os rótulos  (`y` values) são o nome do `parent` de cada arquivo (ou seja, o nome da pasta em que eles estão, que será *bird* or *forest*).

    item_tfms=[Resize(192, method='squish')]

Antes do treinamento, redimensione cada imagem para 192x192 pixels "comprimindo-a" (em vez de cortá-la).

Agora estamos prontos para treinar nosso modelo. O modelo de visão computacional mais rápido e amplamente utilizado é o `resnet18`. Você pode treinar isso em poucos minutos, mesmo em uma CPU! (Em uma GPU, geralmente leva menos de 10 segundos...)

`fastai` vem com um método útil `fine_tune()` que usa automaticamente as melhores práticas para ajustar um modelo pré-treinado, então vamos usá-lo.

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

Geralmente, quando executo isso, vejo 100% de precisão no conjunto de validação (embora possa variar um pouco de execução para execução).

"Ajustar" um modelo significa que estamos começando com um modelo que outra pessoa treinou usando algum outro conjunto de dados (chamado de *modelo pré-treinado*) e ajustando um pouco os pesos para que o modelo aprenda a reconhecer seu conjunto de dados específico . Neste caso, o modelo pré-treinado foi treinado para reconhecer fotos no *imagenet*, um conjunto de dados de visão computacional amplamente utilizado com imagens cobrindo 1.000 categorias. Para obter detalhes sobre o ajuste fino e por que ele é importante, confira o [curso fast.ai gratuito ](https://course.fast.ai/).

## Etapa 3: Use nosso modelo (e construa o seu próprio!)

Vamos ver o que nosso modelo pensa sobre aquele pássaro que baixamos no início:

In [ ]:
is_bird,_,probs = learn.predict(PILImage.create('bird.jpg'))
print(f"This is a: {is_bird}.")
print(f"Probability it's a bird: {probs[0]:.4f}")

Bom trabalho, resnet18. :)

Então, como você pode ver, no espaço de alguns anos, a criação de modelos de classificação de visão computacional passou de “tão difícil que é uma piada” para “trivialmente fácil e gratuito”!

Não é apenas na visão computacional. Graças à aprendizagem profunda, os computadores podem agora fazer muitas coisas que pareciam impossíveis há apenas alguns anos, incluindo [criando obras de arte incríveis](https://openai.com/dall-e-2/), e [explicando piadas](https://www.datanami.com/2022/04/22/googles-massive-new-language-model-can-explain-jokes/). Está avançando tão rápido que até mesmo os especialistas na área têm dificuldade em prever como isso afetará a sociedade nos próximos anos.

Uma coisa é clara -- é importante que todos façamos o nosso melhor para compreender esta tecnologia, caso contrário ficaremos para trás!

Agora é sua vez. Clique em "Copy & Edit" e tente criar seu próprio classificador de imagens usando suas próprias pesquisas de imagens!

Se você gostou disso, considere clicar no botão "upvote" no canto superior direito -- é muito encorajador para nós, autores de cadernos, saber quando as pessoas apreciam nosso trabalho.